Calculate the Call/Put price of a European Option, using the BS formula

In [1]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import statistics as stat
import math as m

In [2]:
from scipy.stats import norm  # Importing the cumulative distribution function (CDF) of the normal distribution

def d_1(S0, K, r, T, vol):
    # Calculate d1, which is an intermediate step in the Black-Scholes formula
    # d1 represents a standardized measure of the difference between the current stock price and the strike price
    # S0: Current stock price
    # K: Strike price
    # r: Risk-free interest rate
    # T: Time to maturity (in years)
    # vol: Volatility of the stock (standard deviation of stock returns)
    
    sol1 = (m.log(S0/K) + (r + 0.5*vol**2)*T) / (vol * np.sqrt(T))  # Using Black-Scholes d1 formula
    return sol1  # Return d1


def d_2(S0, K, r, T, vol):
    # Calculate d2, another intermediate step in the Black-Scholes formula
    # d2 is related to d1 but with a shift in the volatility term
    
    sol2 = (m.log(S0/K) + (r - 0.5*vol**2)*T) / (vol * np.sqrt(T))  # Using Black-Scholes d2 formula
    return sol2  # Return d2


def call_price(S0, K, r, T, vol):
    # Calculate the price of a European call option using the Black-Scholes formula
    # S0: Current stock price
    # K: Strike price
    # r: Risk-free interest rate
    # T: Time to maturity (in years)
    # vol: Volatility of the stock
    
    d1 = d_1(S0, K, r, T, vol)  # Calculate d1
    d2 = d_2(S0, K, r, T, vol)  # Calculate d2
    Nd1 = norm.cdf(d1)  # CDF of standard normal distribution at d1 (probability S0 will be above K)
    Nd2 = norm.cdf(d2)  # CDF of standard normal distribution at d2 (probability discounted to present value)
    
    # Black-Scholes formula for call option price:
    call = S0 * Nd1 - K * m.exp(-r*T) * Nd2  # Present value of stock payoff minus present value of strike price
    return call  # Return the call option price


def put_price(S0, K, r, T, vol):
    # Calculate the price of a European put option using the Black-Scholes formula
    # S0: Current stock price
    # K: Strike price
    # r: Risk-free interest rate
    # T: Time to maturity (in years)
    # vol: Volatility of the stock
    
    d1 = d_1(S0, K, r, T, vol)  # Calculate d1
    d2 = d_2(S0, K, r, T, vol)  # Calculate d2
    Nd1 = norm.cdf(-d1)  # CDF of standard normal distribution at -d1 (probability S0 will be below K)
    Nd2 = norm.cdf(-d2)  # CDF of standard normal distribution at -d2 (probability discounted to present value)
    
    # Black-Scholes formula for put option price:
    put = K * m.exp(-r*T) * Nd2 - S0 * Nd1  # Present value of strike price minus present value of stock payoff
    return put  # Return the put option price

In [3]:
C = call_price(18, 20, 0.08, 3, 0.05)
print("Call option price: £", C)

P = put_price(18, 15, 0.08, 3, 0.05)
print("Put option price: £", P)

Call option price: £ 2.3050780384261547
Put option price: £ 1.2975359338802634e-07


Now, let us construct an algorithm for calculating implied volatility from the BS formula

In [4]:
def fnum(S0, K, r, T, v_i, market_price):
    # fnum: Calculates the difference between the observed market price and the calculated option price
    # This function represents the numerator in the Newton-Raphson method for implied volatility calculation.
    # S0: Current stock price
    # K: Strike price
    # r: Risk-free interest rate
    # T: Time to maturity (in years)
    # v_i: Current volatility estimate
    # market_price: The observed market price of the option
    
    f = market_price - call_price(S0, K, r, T, v_i)  # Difference between market price and Black-Scholes call price
    return f  # Return the difference (residual)


def fdenom(S0, K, r, T, v_i):
    # fdenom: Derivative of the option price with respect to volatility (vega), used in the Newton-Raphson method
    # This function represents the denominator in the Newton-Raphson method for implied volatility calculation.
    # Vega is the sensitivity of the option price to changes in volatility.
    
    fdash = -S0 * (norm.cdf(d_1(S0, K, r, T, v_i))) * np.sqrt(T)  # Approximation of the derivative (vega) using d_1
    return fdash  # Return the derivative (vega)


def NR_algorithm(S0, K, r, T, v0, market_price, iterations):
    # NR_algorithm: Implements the Newton-Raphson method to calculate implied volatility
    # S0: Current stock price
    # K: Strike price
    # r: Risk-free interest rate
    # T: Time to maturity (in years)
    # v0: Initial guess for volatility
    # market_price: The observed market price of the option
    # iterations: Number of iterations to run for the Newton-Raphson method
    
    v_i = v0  # Initialize the volatility estimate with the initial guess (v0)
    
    # Iterate through the Newton-Raphson process to refine the volatility estimate
    for i in range(1, iterations + 1):
        # Update volatility estimate: v_i = v_i - f(v_i) / f'(v_i)
        vnext = v_i - (fnum(S0, K, r, T, v_i, market_price) / fdenom(S0, K, r, T, v_i))
        v_i = vnext  # Set the updated volatility estimate for the next iteration
    
    return v_i  # Return the final volatility estimate after all iterations


In [5]:
implied_volatility = NR_algorithm(29, 28, 0.05, 3/12, 0.15, 2.00, 1000)
print("Implied volatility is: ", implied_volatility)

Implied volatility is:  0.21400828483278975
